# ☁️ Multi-Agent Multimodal Analysis with S3 Vectors Memory

This notebook demonstrates multi-modal content processing using **Strands Agents** with **Amazon S3 Vectors** as the memory backend. This is the production-ready episode in our multi-modal AI processing series.

![s3](https://d2908q01vomqb2.cloudfront.net/da4b9237bacccdf19c0760cab7aec4a8359010b0/2025/07/16/2025-s3-vector-1-vector-overview-1.png)

## What You'll Learn

- **AWS-Native Memory**: Use Amazon S3 Vectors for scalable memory storage
- **Multi-Modal Processing**: Analyze images, documents, and videos with persistent memory
- **Conversation Continuity**: Maintain context across sessions with automatic memory
- **Production Ready**: Enterprise-grade memory solution with AWS integration

## Series Context

This builds upon our previous episode: [Multi-Modal Content Processing with FAISS Memory](https://dev.to/aws/multi-modal-content-processing-with-strands-agent-and-faiss-memory-39hg)

**Key Upgrade**: Moving from local FAISS to AWS-native Amazon S3 Vectors for production-ready memory management.

![a2a](image/s3_memory.png)

## 🤖 Agent Configuration with S3 Vectors Memory


In [ ]:
import boto3
import os
import json
from datetime import datetime

from strands.models import BedrockModel
from strands import Agent
from strands_tools import image_reader, file_read, use_llm
from video_reader import video_reader
from s3_memory import s3_vector_memory  # Our new S3 Vectors memory tool

print("All imports successful!")

### 🔧 Environment Configuration

Configure your Amazon S3 Vectors and AWS settings:

[Tutorial: Getting started with S3 Vectors](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-getting-started.html)

In [ ]:
# S3 Vectors Configuration
os.environ['VECTOR_BUCKET_NAME'] = 'YOUR-S3-BUCKET'  # Your S3 Vector bucket
os.environ['VECTOR_INDEX_NAME'] = 'YOUR-VECTOR-INDEX'        # Your vector index
os.environ['AWS_REGION'] = 'us-east-1'                       # AWS region
os.environ['EMBEDDING_MODEL'] = 'amazon.titan-embed-text-v2:0' # Bedrock embedding model

# User identification for memory isolation
USER_ID = "demo_user_s3_vectors"

print(f"Configuration set for user: {USER_ID}")
print(f"S3 Vector bucket: {os.environ['VECTOR_BUCKET_NAME']}")
print(f"Vector index: {os.environ['VECTOR_INDEX_NAME']}")

### 🚀 Agent Setup with S3 Vectors Memory

Create our multi-modal agent with AWS-native memory capabilities:

In [ ]:
# Model configuration
model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    region="us-east-1"
)

# System prompt for multi-modal processing with memory
MULTIMODAL_SYSTEM_PROMPT = """You are an AI assistant with multi-modal processing capabilities and persistent memory.

Your capabilities:
- **Multi-Modal Analysis**: Process images, documents, videos, and text
- **Persistent Memory**: Remember preferences, previous analyses, and conversation history
- **Context Awareness**: Use memory to provide personalized and contextual responses
- **Continuous Learning**: Build understanding over time through memory accumulation

Memory Usage Guidelines:
- Check for relevant memories before responding
- Store important insights, preferences, and analysis results
- Reference previous conversations when relevant
- Maintain conversation continuity across sessions

When processing content:
1. First retrieve relevant memories for context
2. Analyze the new content thoroughly
3. Store key insights and findings
4. Provide comprehensive responses using both new analysis and memory context
"""

# Create the multi-modal agent with S3 Vectors memory
multimodal_agent = Agent(
    model=model,
    tools=[
        s3_vector_memory,  # Our S3 Vectors memory tool
        image_reader,      # Image processing
        file_read,         # Document processing  
        video_reader,      # Video processing
        use_llm           # Advanced reasoning
    ],
    system_prompt=MULTIMODAL_SYSTEM_PROMPT
)

print("Multi-modal agent with S3 Vectors memory created successfully!")
print("Memory backend: Amazon S3 Vectors")
print("Tools loaded: S3 Memory, Image Reader, File Reader, Video Reader, LLM")

In [ ]:
# 📝 Simulate first interaction - establishing preferences

response1 = multimodal_agent(
    f"""Hello, I'm Elizabeth Fuentes. You can call me Eli, I'm a developer advocate at AWS, I like to work early in the morning, 
    I prefer Italian coffee, and I want to understand what's in images, videos, and documents to improve my day-to-day work. 
    I'm also very interested in artificial intelligence and work in the financial sector.
    
    Please save this information about my preferences for future conversations.
    
    USER_ID: {USER_ID}"""
)

## Memory System Testing

Test our S3 Vectors memory system before processing multi-modal content:

In [ ]:
# Test memory storage
print("Testing S3 Vectors Memory System...\n")

# Store initial preferences
memory_result = s3_vector_memory(
    action="store",
    content="You are interested in AWS architecture, serverless solutions, and multi-modal AI processing. You prefer detailed technical explanations.",
    user_id=USER_ID
)
print(f"Stored preferences: {memory_result['status']}")



In [ ]:
# Test memory retrieval
memory_result = s3_vector_memory(
    action="retrieve",
    query="preferences and interests",
    user_id=USER_ID
)
print(f"Retrieved memories: {memory_result['total_found']} found")


In [ ]:

# List all memories
memory_result = s3_vector_memory(
    action="list",
    user_id=USER_ID
)
print(f"Total memories in system: {memory_result['total_found']}")

## Image Analysis with Memory

Process images with persistent memory for context and insights:

In [ ]:
# Analyze architectural diagram with memory context
print("Analyzing Architectural Diagram with Memory Context...\n")

image_response = multimodal_agent(
    f"""Analyze the architectural diagram in data-sample/diagram.jpg. 
    
    Before analyzing:
    1. Check my memory for any previous architectural discussions or preferences
    2. Use that context to provide a more personalized analysis
    
    After analysis:
    1. Store the key architectural insights you discovered
    2. Note any patterns or technologies that align with my interests
    
    My user ID for memory operations: {USER_ID}
    
    Provide a comprehensive analysis including:
    - Architecture overview and components
    - Technology stack identification
    - Best practices observed
    - Recommendations based on my preferences"""
)

print("Image Analysis Complete!")
print("\n" + "="*80)
print(image_response.message)
print("="*80)

## Document Processing with Memory

Process PDF documents while maintaining conversation context:

In [ ]:
# Process AWS documentation with memory integration
print("Processing AWS Documentation with Memory Integration...\n")

document_response = multimodal_agent(
    f"""Process the document data-sample/Welcome-Strands-Agents-SDK.pdf.
    
    Memory-enhanced processing:
    1. First, retrieve any relevant memories about my interests in AWS, AI, or development tools
    2. Process the document with that context in mind
    3. Store key insights that relate to my interests
    4. Connect the document content to our previous architectural discussion
    
    My user ID: {USER_ID}
    
    Focus on:
    - How Strands Agents relates to the architecture we analyzed
    - Key features that would interest someone focused on AWS and serverless
    - Practical applications for multi-modal AI processing
    - Integration possibilities with AWS services"""
)

print("Document Processing Complete!")
print("\n" + "="*80)
print(document_response.message)
print("="*80)

## Video Analysis with Memory

Process video content with full memory context from previous analyses:

In [ ]:
# Analyze video with comprehensive memory context
print("Analyzing Video with Comprehensive Memory Context...\n")

video_response = multimodal_agent(
    f"""Analyze the video data-sample/moderation-video.mp4 using our full conversation history.
    
    Memory-driven analysis:
    1. Retrieve all relevant memories from our session (architecture, documents, preferences)
    2. Analyze the video content in the context of our previous discussions
    3. Store insights about video content moderation and AI applications
    4. Connect this to the broader multi-modal AI processing theme
    
    My user ID: {USER_ID}
    
    Comprehensive analysis should include:
    - Video content summary and key scenes
    - Technical aspects related to content moderation
    - How this relates to our architectural and Strands Agent discussions
    - Practical applications in AWS/serverless environments
    - Integration possibilities with the technologies we've discussed"""
)

print("Video Analysis Complete!")
print("\n" + "="*80)
print(video_response.message)
print("="*80)

## Memory-Driven Synthesis

Create a comprehensive synthesis using all stored memories:

In [ ]:
# Generate comprehensive synthesis using all memories
print("Generating Memory-Driven Synthesis...\n")

synthesis_response = multimodal_agent(
    f"""Create a comprehensive synthesis of our entire multi-modal processing session.
    
    Memory synthesis process:
    1. Retrieve ALL memories from our session using my user ID: {USER_ID}
    2. Analyze patterns and connections across all processed content
    3. Store this synthesis as a session summary
    4. Provide actionable insights and recommendations
    
    Your synthesis should cover:
    - **Architecture Insights**: Key findings from the diagram analysis
    - **Technology Stack**: Strands Agents capabilities and AWS integration
    - **Multi-Modal Applications**: Practical use cases we've explored
    - **Content Moderation**: Video analysis insights and applications
    - **Memory Benefits**: How S3 Vectors memory enhanced our analysis
    - **Next Steps**: Recommendations for implementing these solutions
    
    Make this a comprehensive technical summary that demonstrates the power of 
    persistent memory in multi-modal AI processing."""
)

print("Comprehensive Synthesis Complete!")
print("\n" + "="*80)
print(synthesis_response.message)
print("="*80)

## Memory System Review

Review what our S3 Vectors memory system has learned:

In [ ]:
# Review all stored memories
print("S3 Vectors Memory System Review\n")

# Get comprehensive memory list
all_memories = s3_vector_memory(
    action="list",
    user_id=USER_ID,
    top_k=50  # Get more memories
)

print(f"\nTotal memories stored: {all_memories['total_found']}")
print("\nMemory Categories Learned:")

# Test different memory retrievals
categories = [
    "architecture and system design",
    "Strands Agents and AI tools", 
    "video content and moderation",
    "preferences and interests",
    "AWS and serverless technologies"
]

for category in categories:
    result = s3_vector_memory(
        action="retrieve",
        query=category,
        user_id=USER_ID,
        top_k=3
    )
    print(f"  {category.title()}: {result['total_found']} relevant memories")

print("\nMemory system successfully captured multi-modal processing insights!")

## Conversation Continuity Test

Test how memory provides conversation continuity across sessions:

In [ ]:
# Simulate a new conversation session with memory context
print("Testing Conversation Continuity with S3 Vectors Memory\n")

continuity_response = multimodal_agent(
    f"""Hi! I'm back for a new session. Can you remind me what we discussed previously 
    and provide some follow-up recommendations based on our multi-modal analysis?
    
    Use my memory (user ID: {USER_ID}) to:
    1. Summarize our previous session
    2. Highlight key insights we discovered
    3. Suggest next steps for implementation
    4. Recommend additional AWS services that would complement our findings
    
    This demonstrates the power of persistent memory in AI conversations!"""
)

print("Conversation Continuity Test Complete!")
print("\n" + "="*80)
print(continuity_response.message)
print("="*80)